In [1]:
#! pip install geemap

     |████████████████████████████████| 476 kB 4.0 MB/s 
     |████████████████████████████████| 1.3 MB 7.6 MB/s 
     |████████████████████████████████| 3.3 MB 19.4 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 98 kB 6.0 MB/s 
     |████████████████████████████████| 1.2 MB 55.5 MB/s 
     |████████████████████████████████| 76 kB 4.1 MB/s 
     |████████████████████████████████| 1.6 MB 21.2 MB/s 
     |████████████████████████████████| 130 kB 60.0 MB/s 
     |████████████████████████████████| 219 kB 51.6 MB/s 
     |████████████████████████████████| 76 kB 4.3 MB/s 
     |████████████████████████████████| 115 kB 52.5 MB/s 
     |████████████████████████████████| 551 kB 65.5 MB/s 
     |████████████████████████████████| 112 kB 47.8 MB/s 
     |████████████████████████████████| 392 kB 44.7 MB/s 
     |████████████████████████████████| 68 kB 6.2 MB/s 
     |████████████████████████████████| 77 kB 5.0 MB/s 
     |██████████████████████

In [2]:
import ee
import geemap

In [3]:
# If you are working with a shape file stored in google drive remove the quotes (""" """)

"""
#connect with drive
from google.colab import drive
drive.mount('/content/drive')
"""

"\n#connect with drive\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [4]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWi7TKdQCDYGvP-Hj7X04rIhkFGxXy1qSGNHFhX68iRudkgdgM2lYco

Successfully saved authorization token.


In [5]:
# https://sites.google.com/site/seriescol/shapes
shape = ee.FeatureCollection('users/dsrestrepo/mpio')

# Cauca = shape.filter(ee.Filter.eq('DPTO_CCDGO', 19))
# Popayan = shape.filter(ee.Filter.eq('MPIO_CCNCT	', 19001))
# Medellin = ee.FeatureCollection('users/dsrestrepo/Medellin')

Popayan = shape.filter(ee.Filter.eq('MPIOS', '19001'))



In [6]:
# Get Dataset Landsat
dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterDate('2019-01-01', '2019-12-31').filterBounds(Popayan)


In [12]:
# Precipitation
Chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate('2019-01-01', '2019-12-31').filterBounds(Popayan)
precipitation = Chirps.select(['precipitation'])
precipitationVis = {
  'min': 1.0,
  'max': 17.0,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000'],
}

Map = geemap.Map()

Map.addLayer(Chirps, precipitationVis, 'Precipitation')
Map.addLayer(shape, {}, 'Popayan')


# Temperature
Modis = ee.ImageCollection('MODIS/006/MOD11A1').filterDate('2019-01-01', '2019-12-31').filterBounds(Popayan)
temperature = Modis.select('LST_Day_1km')
temperatureVis = {
  'min': 13000.0,
  'max': 16500.0,
  'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ],
}
Map.addLayer(temperature.median(), temperatureVis, 'temperature')

# Landsat

rgbPopayan = dataset.select(['B2','B3','B4'])
rgbPopayanVis = {
    'min' : 0.0,
    'max': 0.40
}

Map.addLayer(rgbPopayan.median(), rgbPopayanVis, 'Landsat {432}')

# Shapes
Map.addLayer(shape, {}, 'Popayan')

Map.center_object(Popayan, 13)

Map

Map(center=[2.476705214936409, -76.64337627568842], controls=(WidgetControl(options=['position', 'transparent_…

In [9]:
#Map